In [1]:
import pandas as pd
from dotenv import load_dotenv
import csv

load_dotenv()

from src.orcid_data_utils import collect_professor_data1,extract_department_and_name



Python-dotenv could not parse statement starting at line 2


In [32]:
#Refine raw dataset from scraping to Orcid extraction 
data = pd.read_csv('../../data/raw/staff_data.csv')
names_split = data['Nome Completo'].str.split(' ', n=1, expand=True)

data['Family Name'] = names_split[0]
data['Given Name'] = names_split[1]

data = data[['Family Name', 'Given Name', 'SSD', 'Ruolo']]
data['SSD'] = data['SSD'].str.replace('SSD:', '', regex=False).str.strip()
data[['Department Code', 'Specific Name']] = data['SSD'].apply(extract_department_and_name)
data = data[['Family Name', 'Given Name', 'Department Code', 'Specific Name', 'Ruolo']]

data


,Family Name,Given Name,Department Code,Specific Name,Ruolo
0,ANTONIOTTI,MARCO,(INFO-01/A),Informatica,Professore/ssa ordinario/a
1,ARCELLI,FONTANA FRANCESCA,(IINF-05/A),Sistemi di elaborazione delle informazioni,Professore/ssa ordinario/a
2,BANDINI,STEFANIA,(INFO-01/A),Informatica,Professore/ssa ordinario/a
3,BONIZZONI,PAOLA,(INFO-01/A),Informatica,Professore/ssa ordinario/a
4,CIUCCI,DAVIDE ELIO,(INFO-01/A),Informatica,Professore/ssa ordinario/a
...,...,...,...,...,...
134,SGRO,MATTIA,None,SSD non disponibile,Dottorando/a
135,SHAH,SAHAR,None,SSD non disponibile,Dottorando/a
136,SWEILAM,MOHAMED NASSER HASSAN,None,SSD non disponibile,Dottorando/a
137,TALPINI,JACOPO,None,SSD non disponibile,Dottorando/a


In [33]:
#Traslating Informations
data.rename(columns={'Ruolo': 'Role', 'Specific Name': 'Specific Field'}, inplace=True)

role_translation = {
    'Professore/ssa ordinario/a': 'Full Professor',
    'Professore/ssa associato/a': 'Associate Professor',
    'Ricercatore/rice': 'Researcher',
    'Ricercatore/rice a tempo determinato': 'Fixed-term Researcher',
    'Professore/ssa emerito/a': 'Professor Emeritus',
    'Assegnista di ricerca': 'Research Fellow',
    'Dottorando/a': 'PhD Student'
}
data['Role'].replace(role_translation, inplace=True)

specific_field_translation = {
    'Informatica': 'Computer Science',
    'Sistemi di elaborazione delle informazioni': 'Information Processing Systems',
    'Ricerca operativa': 'Operations Research',
    'SSD non disponibile': 'SSD not available'
}
data['Specific Field'].replace(specific_field_translation, inplace=True)
data.to_csv('../../data/processed/professors_dip_info.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)
data

/var/folders/7y/h7b4hxf568n1y6xlpl8vf6340000gn/T/ipykernel_4709/1971726537.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Role'].replace(role_translation, inplace=True)
/var/folders/7y/h7b4hxf568n1y6xlpl8vf6340000gn/T/ipykernel_4709/1971726537.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values 

,Family Name,Given Name,Department Code,Specific Field,Role
0,ANTONIOTTI,MARCO,(INFO-01/A),Computer Science,Full Professor
1,ARCELLI,FONTANA FRANCESCA,(IINF-05/A),Information Processing Systems,Full Professor
2,BANDINI,STEFANIA,(INFO-01/A),Computer Science,Full Professor
3,BONIZZONI,PAOLA,(INFO-01/A),Computer Science,Full Professor
4,CIUCCI,DAVIDE ELIO,(INFO-01/A),Computer Science,Full Professor
...,...,...,...,...,...
134,SGRO,MATTIA,None,SSD not available,PhD Student
135,SHAH,SAHAR,None,SSD not available,PhD Student
136,SWEILAM,MOHAMED NASSER HASSAN,None,SSD not available,PhD Student
137,TALPINI,JACOPO,None,SSD not available,PhD Student


In [3]:
# Retriving data
professors_df = pd.read_csv('../../data/processed/professors_dip_info.csv')
professors_info, not_found_professors = collect_professor_data1(professors_df)
professors_info_df = pd.DataFrame(professors_info)
professors_info_df.to_csv('../../data/processed/professors_orcid_info.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

print("CSV file created successfully!")


Loop count: 1
Processing professor: MARCO ANTONIOTTI
Added professor: MARCO ANTONIOTTI with organization Università degli Studi di Milano-Bicocca
Loop count: 2
Processing professor: FONTANA FRANCESCA ARCELLI
Added professor: FONTANA FRANCESCA ARCELLI with organization Universita degli Studi di Milano-Bicocca
Loop count: 3
Processing professor: STEFANIA BANDINI
Added professor: STEFANIA BANDINI with organization University of Milano-Bicocca
Loop count: 4
Processing professor: PAOLA BONIZZONI
Added professor: PAOLA BONIZZONI with organization Università degli Studi di Milano-Bicocca
Loop count: 5
Processing professor: DAVIDE CIUCCI
Added professor: DAVIDE CIUCCI with organization University of Milano-Bicocca
Loop count: 6
Processing professor: GABRIELE GIANINI
Added professor: GABRIELE GIANINI with organization Università degli Studi di Milano-Bicocca
Loop count: 7
Processing professor: LEONARDO MARIANI
Added professor: LEONARDO MARIANI with organization University of Milano-Bicocca
Loop

In [6]:
len(not_found_professors)

74

In [7]:
not_found_professors

['VINCENZINA MESSINA',
 "MAURO PEZZE'",
 'RAIMONDO SCHETTINI',
 'ALESSANDRA AGOSTINI',
 'DANIELA BESOZZI',
 "FEDERICO ANTONIO NICCOLO' AMEDEO CABITZA",
 'GIOVANNI DENARO',
 'ALBERTO DENNUNZIO',
 'CLAUDIO FERRETTI',
 'ELISABETTA FERSINI',
 'ALBERTO OTTAVIO LEPORATI',
 'GUGLIELMO LULLI',
 'DANIELA MICUCCI',
 'PAOLO NAPOLETANO',
 'NYSSEN RAFAEL PENALOZA',
 'CHINAGLIA POMELLO LUCIA POMELLO',
 'CLAUDIA RAIBULET',
 'OLIVIERO RIGANELLI',
 'RAFFAELLA RIZZI',
 'DOMENICO GIORGIO SORRENTI',
 'FABIO ANTONIO STELLA',
 'CLAUDIO ZANDRON',
 'ITALO FRANCESCO ZOPPIS',
 'LUCA BERNARDINELLO',
 'GUIDO GIUSEPPE FIORINO',
 'MARCO BUZZELLI',
 'LUIGI CELONA',
 'MARCO CREMASCHI',
 'SIMONE MELZI',
 'FLAVIO PICCOLI',
 'FRANCESCO ANTONIO ARCHETTI',
 'MIRKO PAOLO BARBATO',
 'RAMDHANE FIRAS BEN',
 'FEDERICO BERGAMINI',
 'STEFANIA BOFFA',
 'LUCA CELOTTI',
 'MATTEO ROBERT CHILD',
 'ROBERTO CROTTI',
 'Seyed Mojtaba Daryabari',
 'MANUEL ELIA',
 'YOUNAN GAO',
 'ALESSANDRO GHERARDI',
 'ALESSANDRA GROSSI',
 'LUCA GUGLIELMO

In [4]:
professors_df = pd.read_csv('../../data/processed/professors_dip_info.csv')
professors_df.head(5)

,Family Name,Given Name,Department Code,Specific Field,Role
0,ANTONIOTTI,MARCO,(INFO-01/A),Computer Science,Full Professor
1,ARCELLI,FONTANA FRANCESCA,(IINF-05/A),Information Processing Systems,Full Professor
2,BANDINI,STEFANIA,(INFO-01/A),Computer Science,Full Professor
3,BONIZZONI,PAOLA,(INFO-01/A),Computer Science,Full Professor
4,CIUCCI,DAVIDE,(INFO-01/A),Computer Science,Full Professor
